In [146]:
import numpy as np
import pandas as pd
from sodapy import Socrata

In [147]:
#Download the data and create a dataframe
client = Socrata("data.montgomeryal.gov", None)
results = client.get("pjb8-sd6v", limit=10000)
raw_data = pd.DataFrame.from_records(results)

In [148]:
#Make a copy of raw_data and only the columns we need
data = raw_data.drop(['annualsalaryytd', 'name', 'otherpayamt', 'otherpaydesc', 'overtimeamt', 'hire_date'], axis=1)

In [149]:
#Convert necessary columns to numeric
num_cols = pd.Index(['annual_salary', 'step'])
data[num_cols] = data[num_cols].apply(pd.to_numeric, errors='coerce')

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2405 entries, 0 to 2404
Data columns (total 6 columns):
annual_salary     2405 non-null float64
department        2405 non-null object
grade             2405 non-null object
position_title    2405 non-null object
positiontype      2405 non-null object
step              2405 non-null int64
dtypes: float64(1), int64(1), object(4)
memory usage: 112.8+ KB


In [150]:
# Create a class to select numerical or categorical columns 
# since Scikit-Learn doesn't handle DataFrames yet
from sklearn.base import BaseEstimator, TransformerMixin
class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names].values

In [151]:
# Definition of the CategoricalEncoder class, copied from PR #9151.
# CategoricalEncoder is not to be released until version 0.20 of sklearn

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.utils import check_array
from sklearn.preprocessing import LabelEncoder
from scipy import sparse

class CategoricalEncoder(BaseEstimator, TransformerMixin):
    """Encode categorical features as a numeric array.
    The input to this transformer should be a matrix of integers or strings,
    denoting the values taken on by categorical (discrete) features.
    The features can be encoded using a one-hot aka one-of-K scheme
    (``encoding='onehot'``, the default) or converted to ordinal integers
    (``encoding='ordinal'``).
    This encoding is needed for feeding categorical data to many scikit-learn
    estimators, notably linear models and SVMs with the standard kernels.
    Read more in the :ref:`User Guide <preprocessing_categorical_features>`.
    Parameters
    ----------
    encoding : str, 'onehot', 'onehot-dense' or 'ordinal'
        The type of encoding to use (default is 'onehot'):
        - 'onehot': encode the features using a one-hot aka one-of-K scheme
          (or also called 'dummy' encoding). This creates a binary column for
          each category and returns a sparse matrix.
        - 'onehot-dense': the same as 'onehot' but returns a dense array
          instead of a sparse matrix.
        - 'ordinal': encode the features as ordinal integers. This results in
          a single column of integers (0 to n_categories - 1) per feature.
    categories : 'auto' or a list of lists/arrays of values.
        Categories (unique values) per feature:
        - 'auto' : Determine categories automatically from the training data.
        - list : ``categories[i]`` holds the categories expected in the ith
          column. The passed categories are sorted before encoding the data
          (used categories can be found in the ``categories_`` attribute).
    dtype : number type, default np.float64
        Desired dtype of output.
    handle_unknown : 'error' (default) or 'ignore'
        Whether to raise an error or ignore if a unknown categorical feature is
        present during transform (default is to raise). When this is parameter
        is set to 'ignore' and an unknown category is encountered during
        transform, the resulting one-hot encoded columns for this feature
        will be all zeros.
        Ignoring unknown categories is not supported for
        ``encoding='ordinal'``.
    Attributes
    ----------
    categories_ : list of arrays
        The categories of each feature determined during fitting. When
        categories were specified manually, this holds the sorted categories
        (in order corresponding with output of `transform`).
    Examples
    --------
    Given a dataset with three features and two samples, we let the encoder
    find the maximum value per feature and transform the data to a binary
    one-hot encoding.
    >>> from sklearn.preprocessing import CategoricalEncoder
    >>> enc = CategoricalEncoder(handle_unknown='ignore')
    >>> enc.fit([[0, 0, 3], [1, 1, 0], [0, 2, 1], [1, 0, 2]])
    ... # doctest: +ELLIPSIS
    CategoricalEncoder(categories='auto', dtype=<... 'numpy.float64'>,
              encoding='onehot', handle_unknown='ignore')
    >>> enc.transform([[0, 1, 1], [1, 0, 4]]).toarray()
    array([[ 1.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,  0.],
           [ 0.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.]])
    See also
    --------
    sklearn.preprocessing.OneHotEncoder : performs a one-hot encoding of
      integer ordinal features. The ``OneHotEncoder assumes`` that input
      features take on values in the range ``[0, max(feature)]`` instead of
      using the unique values.
    sklearn.feature_extraction.DictVectorizer : performs a one-hot encoding of
      dictionary items (also handles string-valued features).
    sklearn.feature_extraction.FeatureHasher : performs an approximate one-hot
      encoding of dictionary items or strings.
    """

    def __init__(self, encoding='onehot', categories='auto', dtype=np.float64,
                 handle_unknown='error'):
        self.encoding = encoding
        self.categories = categories
        self.dtype = dtype
        self.handle_unknown = handle_unknown

    def fit(self, X, y=None):
        """Fit the CategoricalEncoder to X.
        Parameters
        ----------
        X : array-like, shape [n_samples, n_feature]
            The data to determine the categories of each feature.
        Returns
        -------
        self
        """

        if self.encoding not in ['onehot', 'onehot-dense', 'ordinal']:
            template = ("encoding should be either 'onehot', 'onehot-dense' "
                        "or 'ordinal', got %s")
            raise ValueError(template % self.handle_unknown)

        if self.handle_unknown not in ['error', 'ignore']:
            template = ("handle_unknown should be either 'error' or "
                        "'ignore', got %s")
            raise ValueError(template % self.handle_unknown)

        if self.encoding == 'ordinal' and self.handle_unknown == 'ignore':
            raise ValueError("handle_unknown='ignore' is not supported for"
                             " encoding='ordinal'")

        X = check_array(X, dtype=np.object, accept_sparse='csc', copy=True)
        n_samples, n_features = X.shape

        self._label_encoders_ = [LabelEncoder() for _ in range(n_features)]

        for i in range(n_features):
            le = self._label_encoders_[i]
            Xi = X[:, i]
            if self.categories == 'auto':
                le.fit(Xi)
            else:
                valid_mask = np.in1d(Xi, self.categories[i])
                if not np.all(valid_mask):
                    if self.handle_unknown == 'error':
                        diff = np.unique(Xi[~valid_mask])
                        msg = ("Found unknown categories {0} in column {1}"
                               " during fit".format(diff, i))
                        raise ValueError(msg)
                le.classes_ = np.array(np.sort(self.categories[i]))

        self.categories_ = [le.classes_ for le in self._label_encoders_]

        return self

    def transform(self, X):
        """Transform X using one-hot encoding.
        Parameters
        ----------
        X : array-like, shape [n_samples, n_features]
            The data to encode.
        Returns
        -------
        X_out : sparse matrix or a 2-d array
            Transformed input.
        """
        X = check_array(X, accept_sparse='csc', dtype=np.object, copy=True)
        n_samples, n_features = X.shape
        X_int = np.zeros_like(X, dtype=np.int)
        X_mask = np.ones_like(X, dtype=np.bool)

        for i in range(n_features):
            valid_mask = np.in1d(X[:, i], self.categories_[i])

            if not np.all(valid_mask):
                if self.handle_unknown == 'error':
                    diff = np.unique(X[~valid_mask, i])
                    msg = ("Found unknown categories {0} in column {1}"
                           " during transform".format(diff, i))
                    raise ValueError(msg)
                else:
                    # Set the problematic rows to an acceptable value and
                    # continue `The rows are marked `X_mask` and will be
                    # removed later.
                    X_mask[:, i] = valid_mask
                    X[:, i][~valid_mask] = self.categories_[i][0]
            X_int[:, i] = self._label_encoders_[i].transform(X[:, i])

        if self.encoding == 'ordinal':
            return X_int.astype(self.dtype, copy=False)

        mask = X_mask.ravel()
        n_values = [cats.shape[0] for cats in self.categories_]
        n_values = np.array([0] + n_values)
        indices = np.cumsum(n_values)

        column_indices = (X_int + indices[:-1]).ravel()[mask]
        row_indices = np.repeat(np.arange(n_samples, dtype=np.int32),
                                n_features)[mask]
        data = np.ones(n_samples * n_features)[mask]

        out = sparse.csc_matrix((data, (row_indices, column_indices)),
                                shape=(n_samples, indices[-1]),
                                dtype=self.dtype).tocsr()
        if self.encoding == 'onehot-dense':
            return out.toarray()
        else:
            return out

In [152]:
from sklearn.pipeline import Pipeline, FeatureUnion

num_attribs = ["step"]
cat_attribs = ["department", "grade", "position_title", "positiontype"]

num_pipeline = Pipeline([
        ('selector', DataFrameSelector(num_attribs)),
        ('imputer', Imputer(strategy="median")),
        ('std_scaler', StandardScaler()),
    ])

cat_pipeline = Pipeline([
        ('selector', DataFrameSelector(cat_attribs)),
        ('cat_encoder', CategoricalEncoder(encoding="onehot-dense")),
    ])

full_pipeline = FeatureUnion(transformer_list=[
        ("num_pipeline", num_pipeline),
        ("cat_pipeline", cat_pipeline),
    ])

In [153]:
#Select only the necessary columns
X = data.drop(['annual_salary'], axis=1)
y = data[["annual_salary"]]

In [154]:
X_transformed = full_pipeline.fit_transform(X)

In [155]:
#Split the dataset
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_transformed, y, test_size=0.20, random_state=42)

In [156]:
#Fit the model to the training data
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train.values)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [157]:
#Check the mean squared error of the training data
from sklearn.metrics import mean_squared_error
lin_mse = mean_squared_error(y_train, lin_reg.predict(X_train))
lin_rmse = np.sqrt(lin_mse)
lin_rmse

1003.2036467689019

In [158]:
#Now check the mean squared error of the training data.  What the!?
predictions = lin_reg.predict(X_test)
lin_mse = mean_squared_error(y_test, predictions)
lin_rmse = np.sqrt(lin_mse)
lin_rmse

62221675860410.125

In [159]:
#The 18th prediction is way off
np.int64(predictions[:18,:])

array([[          47003],
       [          42783],
       [          51699],
       [          38749],
       [          32330],
       [          40761],
       [          57347],
       [          34999],
       [          40954],
       [          65963],
       [          18003],
       [          55162],
       [          32330],
       [          34351],
       [          39750],
       [          39750],
       [          30862],
       [839562765827899]], dtype=int64)

In [164]:
#Check the first 18 y_test values
y_test.annual_salary[:18]

2179    46946.39
1525    43117.57
1377    50968.74
2372    38316.10
1125    32318.00
1697    40676.69
1973    57911.24
1027    34567.10
929     40894.26
210     63243.98
1517    18000.00
1724    54436.30
927     32318.00
2028    34599.34
1908    39456.35
56      39456.35
1938    29955.95
218     50251.76
Name: annual_salary, dtype: float64

In [160]:
#The 18th prediction should be closer to this number
y_test.annual_salary[17:18]

218    50251.76
Name: annual_salary, dtype: float64

In [161]:
#The data for this record [218] doesn't look unusual
data.iloc[218]

annual_salary                50251.8
department        CITY INVESTIGATION
grade                            A08
position_title     CITY INVESTIGATOR
positiontype               Full Time
step                               4
Name: 218, dtype: object

In [162]:
#Comparison of similar nearby records
X.iloc[217:220]

,department,grade,position_title,positiontype,step
217,LIBRARY,A02,LIBRARY ASSISTANT I,Full Time,2
218,CITY INVESTIGATION,A08,CITY INVESTIGATOR,Full Time,4
219,FIRE DEPARTMENT,PSC,FIRE LIEUTENANT,Full Time,5


In [163]:
#All the predictions that are way off
predictions[predictions < 0], predictions[predictions > 150000]

(array([ -1.11749936e+14,  -2.18609563e+13,  -3.01885247e+13,
         -5.74222347e+13,  -1.05990542e+14,  -5.84401447e+12,
         -4.59411108e+13,  -8.46318504e+13,  -4.03443478e+13,
         -2.11014104e+13,  -2.95518834e+11,  -4.04503420e+13,
         -7.02792413e+13,  -5.84401449e+12,  -4.04503420e+13,
         -2.18609563e+13,  -4.04503420e+13,  -5.84401445e+12,
         -5.84401445e+12,  -4.96538202e+12]),
 array([  8.39562766e+14,   7.72504222e+12,   6.88653902e+13,
          9.30614592e+13,   2.96495886e+13,   4.96641012e+14,
          2.09965187e+13,   9.15286058e+14,   6.86404834e+13,
          4.70044339e+13,   3.15838781e+13]))